In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.linalg import sqrtm, expm
PI = np.pi

# Quantum Eigenvalue Transform Simulation
## The object of QEVT is to apply a polynomial transform on the eigenvalues of a Hermitian matrix.

It works by embedding a Hermitian $H$ into a larger unitary operator, and then applying the transfom the this unitary.  
The example shown in the paper uses the following block encoding
$$U = \begin{bmatrix}H & \sqrt{I-H^2}\\ \sqrt{I-H^2} & H\end{bmatrix}$$
This is implemented in the function block_encoding(H)

In [ ]:
def block_encoding(H: np.ndarray):
    n = len(H[0])
    I = np.eye(n)
    sqrt_I_minus_H2 = sqrtm(I - (H @ H))
    
    U = np.block([
        [H, sqrt_I_minus_H2],
        [sqrt_I_minus_H2, -H]
    ])
    
    return U

We also use a projector-controlled phase shift, this operatos applies a Z rotation on each eigenspace independetly, and is given by the formula $$Π_\phi=e^{i2\phiΠ}$$
with $Π=\ket0\bra0$

In [ ]:
def PI_phi(phi, n) -> np.ndarray: 
    I_n = np.eye(n)
    Pi = np.kron(np.array([[1, 0], [0, 0]]), I_n) # Π = ∣0⟩⟨0∣, adjusted to dimension n
    return expm(1j * 2 * phi * Pi) # exp(2iφΠ)

We now interleave these two operators to obtain the final QEVT sequence, depending on the parity of $d$.
    
For even $d$ $$U_{\vec{\phi}}=\prod_{k=1}^{d/2} Π_{{\phi}_{2k-1}}U^{\dagger}Π_{{\phi}_{2k}}U$$  
For odd $d$ $$U_{\vec{\phi}}=Π_{{\phi}_1}U\begin{bmatrix}\prod_{k=1}^{(d-1)/2} Π_{{\phi}_{2k}}U^{\dagger}Π_{{\phi}_{2k+1}}U\end{bmatrix}$$  
in both cases, this delivers a unitary of the form
$$\begin{bmatrix}Poly(H) & . \\ . & .\end{bmatrix}$$
Where $Poly(H)$ is the polynomial transformation of the eigenvalues of $H$

In [ ]:
def QEVT(H, phi):
    n = len(H[0])
    d = len(phi)
    
    U = block_encoding(H)
    U_dagger = U.conj().T
    
    U_phi = np.eye(2 * n, dtype=complex)
    
    if d % 2 == 0: #first case, d even
        k = 0
        while k < d:
            U_phi = U_phi @ PI_phi(phi[k],n) @ U_dagger @ PI_phi(phi[k+1],n) @ U
            k+=2
    else: #second case, d odd
        U_phi = PI_phi(phi[0],n) @ U
        k = 1
        while k < d:
            U_phi = U_phi @ PI_phi(phi[k],n) @ U_dagger @ PI_phi(phi[k+1],n) @ U
            k+=2
    
    Poly_H = U_phi[:n, :n]
    
    return Poly_H